- Goal
    - Create synthetic data using IDM
    - Learn parameters of that data using particle filtering
    - Aliter 7 Feb: Learn using CEM idea, fitness function and then distb and then sample
- Learning
    - Need at least 2 vehicles so that there is a neighbor in the front
    - Otherwise src/1d/driver/lane_follower_driver errors
    - That is why the `AutomotiveDrivingModels/doc/1DMobius` stuff does 
    not work with 1 car only (needs at least 2 cars)
    - The tutorial does not work with `gen_straight_roadway` because that
    generates a roadway of type AutomotiveDrivingModels.roadway as opposed to
    AutomotiveDrivingModels.StraightRoadway
- Open question
    - IDM won't work unless there is a car in front (errors saynig nothing in sight)
    - We are focusing on learning the params of the second car here. Is that sound sensible?
- Flow of code
    - Call the required `usings`
    - Define functions required
    - Actual running things
        - Generate true trajectory
        - Generate a set of particles uniformly between sensible range of values
        - Compute fitness, sort and select numtop
        - Fit a distribution over this
        - Resample particles

In [1]:
using AutomotiveDrivingModels
using AutoViz
using Reel
using Interact
using StatsBase # For random particle generation

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [5]:
# 1 lane, 1000 m roadway
# roadway = gen_straight_roadway(1,1000.0) # Does not work with the below because AutomotiveDrivingModels.Roadway
roadway = StraightRoadway(1000.0); # AutomotiveDrivingModels.StraightRoadway type

In [ ]:
scene = Scene1D()
push!(scene, Entity(State1D(10.0,  8.0), VehicleDef(), 1))
push!(scene, Entity(State1D(50.0, 12.5), VehicleDef(), 2))

# cam = StaticCamera(VecE2(100.0,0.0), 4.75)
# overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true,
#         pos=VecE2(veh.state.s-0.7, 3)) for veh in scene];
# render(scene, roadway, overlays, cam=cam, canvas_height=100)

In [ ]:
# Example of fitting data to a distribution
d = Normal() # Create a normal distribution
x = rand(d, 100) # Generate 100 samples from this distrib
fit(Normal, x)

In [6]:
# Function: Initialize the scene
function init_scene()
    scene = Scene1D()
    push!(scene, Entity(State1D(10.0, 8.0), VehicleDef(), 1))
    push!(scene, Entity(State1D(50.0, 12.5), VehicleDef(), 2))
    return scene
end

init_scene (generic function with 1 method)

In [7]:
# Function: Generate trajectory
    # Calls init_scene
function gen_traj(param_vdes;nticks=100,timestep=0.1)
    scene = init_scene()
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(v_des=12.0) # always produce zero acceleration
    models[2] = IntelligentDriverModel(v_des=param_vdes)

    # Simulate for nticks (default 100) time steps
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    # Extract the position and velocity of nticks timesteps
    # X stores this as 2d array. Timestep is the row, col 1 is pos,vel is pos2
    n_cars = scene.n
    n_ticks = nticks
    X = Array{Float64}(n_ticks, 1)

    for t in 1:n_ticks
        f = rec.frames[n_ticks - t + 1]
        
        # BAD: 2 cars in scene therefore loop has only 1 elem
        for c in 2:scene.n #TODO: THIS IS A PITFALL AND IS HARDCODED
            s = f.entities[c].state
            X[t, 1] = s.s #position
        end
    end
    return X
end

gen_traj (generic function with 1 method)

$\sum_{t=1}^{t=end}\mid truePos_t - hallucinatedPos_t \mid$

In [8]:
# Function: Calculate abs pos error at every timestep
#    Some up those errors to create an error over entire traj
function calc_traj_error(traj1,traj2)
    return sum(abs.(traj1-traj2))
end

calc_traj_error (generic function with 1 method)

In [9]:
# Generate the true trajectory
true_traj = gen_traj(20.0);

In [10]:
# Set of num_p particles in the range p_start_val to p_end_val
# Each particle represents a desired velocity that parametrizes
# the IDM model
p_start_val = 10.0
p_end_val = 30.0
num_p = 50
p_vdes = sample(p_start_val:p_end_val, num_p, replace = true);

In [12]:
# Loop over the set of particles and calculate fitness
p_traj_error = Array{Float64}(length(p_vdes))
for (i,proposed_vdes) in enumerate(p_vdes)
    proposed_traj = gen_traj(proposed_vdes)
    p_traj_error[i] = calc_traj_error(true_traj,proposed_traj)
end

In [13]:
# We don't care about the actual error values
# We just care about the numtop lowest of them

# Collect the indices of the lowest to highest elems
#    in the traj error vector
sortedidx = sortperm(p_traj_error)

# Select the numtop from the original particles
numtop = 10
new_p_vdes = p_vdes[sortedidx[1:numtop]]

10-element Array{Float64,1}:
 20.0
 20.0
 20.0
 20.0
 21.0
 21.0
 21.0
 21.0
 19.0
 19.0

In [15]:
# Use the numtop particles to fit a Gaussian
p_dist = fit(Normal,new_p_vdes)

Distributions.Normal{Float64}(μ=20.2, σ=0.7483314773547883)

In [20]:
# Sample num_p new particles from this fitted distb
p_v_des = round.(rand(p_dist,num_p))

50-element Array{Float64,1}:
 19.0
 21.0
 21.0
 22.0
 21.0
 19.0
 21.0
 20.0
 21.0
 19.0
 20.0
 21.0
 18.0
  ⋮  
 20.0
 19.0
 19.0
 20.0
 21.0
 21.0
 20.0
 20.0
 20.0
 21.0
 20.0
 20.0

## Particle Filter Idea tabled for now

In [ ]:
# Hallucinate a step using all the particles
raise()
for (i,my_v_des) in enumerate(p_vdes)
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(v_des=12.0) # always produce zero acceleration
    models[2] = IntelligentDriverModel(v_des=my_v_des) # default IDM with a desired speed of 12 m/s

    nticks = 0.1
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)
end

In [ ]:
# Compare every particles hallucination to
# the master trajectory (what actually happened)
# and assign a weight

In [ ]:
models = Dict{Int, LaneFollowingDriver}()
models[1] = IntelligentDriverModel(v_des=12.0) # always produce zero acceleration
models[2] = IntelligentDriverModel(v_des=20.0) # default IDM with a desired speed of 12 m/s

nticks = 5
timestep = 0.1
rec = QueueRecord(Vehicle1D, nticks+1, timestep)
simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)
@show length(rec.frames)

In [ ]:
fieldnames(rec.frames[1].entities[1].state)

In [ ]:
fieldnames(IntelligentDriverModel)

In [ ]:
# From djp chmm bitbucket. src/toy_driving_data.jl
n_cars = scene.n
n_ticks = nticks
X = Array{Float64}(n_cars, n_ticks, 3)

for t in 1:n_ticks
    f = rec.frames[n_ticks - t + 1]
    for c in 1:scene.n
        s = f.entities[c].state
        X[c, t, 1] = s.s #position
        X[c, t, 2] = s.v #speed
    end
end

Ideas:
- Hallucination and weighting can happen in same iteration
- Should we correct the position of the car to what actually happened after every timestep

In [ ]:
X

In [ ]:
using Interact
@manipulate for frame_index in 1 : nframes(rec)
    render(rec[frame_index-nframes(rec)], roadway, cam=cam, canvas_height=100)
end

# 2D experimentation

In [ ]:
roadway = gen_straight_roadway(2,1000.0);

In [ ]:
scene = Scene1D()
push!(scene, Entity(State1D(10.0,  8.0), VehicleDef(), 1))
push!(scene, Entity(State1D(50.0, 12.5), VehicleDef(), 2))

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, pos=VecE2(veh.state.s-0.7, 3)) for veh in scene]
render(scene, roadway, overlays, cam=cam, canvas_height=100)

In [ ]:
models = Dict{Int64, DriverModel}()
models[1] = Tim2DDriver(0.1) # always produce zero acceleration
models[2] = Tim2DDriver(0.1) # default IDM with a desired speed of 12 m/s

nticks = 100
timestep = 0.1
rec = QueueRecord(Vehicle1D, nticks+1, timestep)
simulate!(rec, scene, roadway, models, nticks)